# Assignment for Week 4
# Find best place to open a restaurant

The aim of this project is to find best location and type of a new resturant that the client would like to open in Florence.
In order to do this I divided Florence basing on Postal Code of different areas, I plotted them and found nearby venues. 
Then I analyzed data to have the answers I was searching for and I wrote my conclusions.

## Building DataFrame
Since Postal Codes of Florence ranges between 50121 and 50145 I build dataframe finding Latitude and Longitude for each different Postal Code.

In [1]:
import sys
#!{sys.executable} -m pip install geocoder
import geocoder

def latlon(postal_code):
    global latitude, longitude
    lat_lng_coords = None

    while(lat_lng_coords is None):
      g = geocoder.arcgis('{}, Firenze, Italy'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

    
    return latitude,longitude

In [2]:
import pandas as pd
import sys
df=pd.DataFrame(columns=['CAP','Latitude','Longitude'])

for cp,i in zip(range(50121,50145),range(0,50145-50121)):
    latlon(cp)
    df.loc[i,'CAP']=cp
    df.loc[i,'Latitude']=latitude
    df.loc[i,'Longitude']=longitude
    sys.stdout.write('\r'+'Done '+str(i+1)+' of '+str(50145-50121))
    sys.stdout.flush()
print('\nAll data were scraped!')
df.head()

Done 24 of 24
All data were scraped!


,CAP,Latitude,Longitude
0,50121,43.7744,11.2645
1,50122,43.7714,11.2616
2,50123,43.7743,11.2472
3,50124,43.7497,11.2241
4,50125,43.7479,11.2585


The next step is to plot these points in a map using Folium library:

In [3]:
import sys
#!{sys.executable} -m pip install folium


import folium
map_florence = folium.Map(location=[43.7696, 11.2558], zoom_start=12)

# add markers to map
for lat, lng,cp in zip(df['Latitude'], df['Longitude'],df['CAP']):
    label = '{}'.format(cp)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_florence)  
    
map_florence

## Get nearby venues from FourSquare
Now it's the time to retrieve data from FourSquare. In this section I'll get 500 nearby venues for each point that i plot in the above map.

In [4]:
import requests
CLIENT_ID = 'AMQOC0PAD201KRERHODMBXYHTVPMF1UMLLO5UDZVUB4NWOPH' # your Foursquare ID
CLIENT_SECRET = 'HDTMQKXWJD3ISPTMZPT0KL1GUYYFT1ABL4GVVIXBWL0BDDX4' # your Foursquare Secret
VERSION = '20180604'
intent='food'
LIMIT=10000
price='1'
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng, i in zip(names, latitudes, longitudes, range(1,len(names))):
        
        sys.stdout.write('\r'+'Getting nearby values for '+str(i)+' of '+str(len(names)-1))
        sys.stdout.flush()
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&intent={}&price={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            intent,
            price
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['CAP', 
                  'CAP Latitude', 
                  'CAP Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [5]:
Florence_venues = getNearbyVenues(names=df['CAP'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )
Florence_venues.head()

Getting nearby values for 23 of 23

,CAP,CAP Latitude,CAP Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,50121,43.774415,11.264521,Panini Toscani,43.773110,11.257390,Sandwich Place
1,50121,43.774415,11.264521,All'Antico Vinaio,43.768511,11.257318,Sandwich Place
2,50121,43.774415,11.264521,Pino's Sandwiches,43.770501,11.261799,Sandwich Place
3,50121,43.774415,11.264521,Sandwichic,43.777179,11.256242,Sandwich Place
4,50121,43.774415,11.264521,Vecchio Forno,43.777130,11.256177,Bakery


In [6]:
Florence_venues=Florence_venues[Florence_venues['Venue Category'].str.contains('estaurant')]
Florence_venues.shape

(34, 7)

Only 34 restaurants are present in the data I downloaded with Foursquare API.This is a very small number if compared with the real situation.
I decided then to made a python script for scraping data from Tripadvisor. With this script (it took a while to run) I was able to scrape a lot of useful data for this project like price, kind of cousine, ranking, reviews etc. 
Below I attach (in small red) the script I made and run.

<font size="5" color="red"><a href='https://drive.google.com/file/d/1qc3kheIpHTFvGofYXJ7q1OoCn0lhQqlL/view?usp=sharing'>Click to download the scraper py file!</font>

Filtering found venues selecting only restaurants:

In [7]:
import pandas as pd
import requests
from io import StringIO

orig_url='https://drive.google.com/file/d/14ejjZZs9VP6T5680EmRMrOLf5mnVeqL5/view?usp=sharing'

file_id = orig_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
data=pd.read_csv(csv_raw).iloc[:,1:]
data.head()

,Restaurant,Ranking,Reviews,Price,Range_price,Cousine_type,Address,Phone,Site,Area,Near,Url
0,CaffÃ¨ Italiano,"4,0",1.083 recensioni,â¬â¬-â¬â¬â¬,NaN,"Italiana, Pizza, Mediterranea, Toscana, Italia...","Via Isola delle Stinche, 11R-13R, 50122, Firen...",+39 055 289080,NaN,Duomo,"0,3 km da Piazza del Duomo",https://www.tripadvisor.it/Restaurant_Review-g...
1,Gustarium,"5,0",1.990 recensioni,â¬,NaN,"Pizza, Mediterranea, Italiana, Fast food, Salu...","Via Dei Cimatori 24/r, 50122, Firenze Italia",+39 055 283469,NaN,Duomo,"0,1 km da Palazzo Vecchio",https://www.tripadvisor.it/Restaurant_Review-g...
2,Degusteria Italiana agli Uffizi,"5,0",689 recensioni,â¬â¬-â¬â¬â¬,NaN,"Mediterranea, Salutistica, Wine Bar, Italiana,...",Via Lambertesca 7r A Pochi Metri Dal Museo Uff...,+39 055 493 9867,NaN,Centro storico di Firenze,"0,1 km da Gallerie Degli Uffizi",https://www.tripadvisor.it/Restaurant_Review-g...
3,180 Grammi Burgers & Beers,"5,0",80 recensioni,NaN,NaN,Italiana,"Via Pietrapiana 11/ R Via Pietrapiana 11/ R, 5...",+39 055 386 0476,NaN,Santa Croce,"0,6 km da Piazza del Duomo",https://www.tripadvisor.it/Restaurant_Review-g...
4,Pitti Express,"5,0",85 recensioni,â¬,NaN,"CaffÃ¨, Wine Bar, Toscana, Salutistica, Fast f...","Via Romana 40/r, 50125, Firenze Italia",+39 055 627 8178,NaN,Oltrarno,"0,6 km da Ponte Vecchio",https://www.tripadvisor.it/Restaurant_Review-g...


Finding Latitude and Longitude for each venue:

In [8]:
def latlon_comp_addr(complete_address):
    global latitude, longitude
    lat_lng_coords = None

    while(lat_lng_coords is None):
        #g = geocoder.mapquest('{}'.format(complete_address), key='Wxgd9xJHn8bLtY8h7xuEANsfhGN7n0GH')
        g = geocoder.arcgis('{}'.format(complete_address))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

    
    return latitude,longitude

columns=list(data.columns)
columns.extend(['Latitude','Longitude'])

Since the dimension of data is large, in order to run the notebook faster, I left the original code as comment and after the first run I exported as csv and then reimported.

In [73]:
'''
df=data.iloc[:,:]
df['Latitude']=''
df['Longitude']=''

for addr,i in zip(data['Address'],data.index):
    latlon_comp_addr(addr)
    df.loc[i,'Latitude']=latitude
    df.loc[i,'Longitude']=longitude
    sys.stdout.write('\r'+'Done '+str(i+1)+' of '+str(len(data.index))
    sys.stdout.flush()
print('\nAll data were scraped!')
'''
orig_url='https://drive.google.com/file/d/1gSIfOOOkAXqzlYa5c1RntLsGh2elxvUL/view?usp=sharing'

file_id = orig_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
df=pd.read_csv(csv_raw).iloc[:,1:]

df.head()


,Restaurant,Ranking,Reviews,Price,Range_price,Cousine_type,Address,Phone,Site,Area,Near,Url,Latitude,Longitude
0,CaffÃÂ¨ Italiano,"4,0",1.083 recensioni,Ã¢ÂÂ¬Ã¢ÂÂ¬-Ã¢ÂÂ¬Ã¢ÂÂ¬Ã¢ÂÂ¬,NaN,"Italiana, Pizza, Mediterranea, Toscana, Italia...","Via Isola delle Stinche, 11R-13R, 50122, Firen...",+39 055 289080,NaN,Duomo,"0,3 km da Piazza del Duomo",https://www.tripadvisor.it/Restaurant_Review-g...,43.770131,11.260304
1,Gustarium,"5,0",1.990 recensioni,Ã¢ÂÂ¬,NaN,"Pizza, Mediterranea, Italiana, Fast food, Salu...","Via Dei Cimatori 24/r, 50122, Firenze Italia",+39 055 283469,NaN,Duomo,"0,1 km da Palazzo Vecchio",https://www.tripadvisor.it/Restaurant_Review-g...,43.770654,11.255570
2,Degusteria Italiana agli Uffizi,"5,0",689 recensioni,Ã¢ÂÂ¬Ã¢ÂÂ¬-Ã¢ÂÂ¬Ã¢ÂÂ¬Ã¢ÂÂ¬,NaN,"Mediterranea, Salutistica, Wine Bar, Italiana,...",Via Lambertesca 7r A Pochi Metri Dal Museo Uff...,+39 055 493 9867,NaN,Centro storico di Firenze,"0,1 km da Gallerie Degli Uffizi",https://www.tripadvisor.it/Restaurant_Review-g...,43.768834,11.254358
3,180 Grammi Burgers & Beers,"5,0",80 recensioni,NaN,NaN,Italiana,"Via Pietrapiana 11/ R Via Pietrapiana 11/ R, 5...",+39 055 386 0476,NaN,Santa Croce,"0,6 km da Piazza del Duomo",https://www.tripadvisor.it/Restaurant_Review-g...,43.771410,11.265810
4,Pitti Express,"5,0",85 recensioni,Ã¢ÂÂ¬,NaN,"CaffÃÂ¨, Wine Bar, Toscana, Salutistica, Fast...","Via Romana 40/r, 50125, Firenze Italia",+39 055 627 8178,NaN,Oltrarno,"0,6 km da Ponte Vecchio",https://www.tripadvisor.it/Restaurant_Review-g...,43.763190,11.244220


In [45]:
venues_map = folium.Map(location=[43.7696, 11.2558], zoom_start=12)

# add markers to map
for lat, lng,cp in zip(df['Latitude'], df['Longitude'],df['Restaurant']):
    label = '{}'.format(cp)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#A79AFF',
        fill=True,
        fill_color='#ECD4FF',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map)  
    
venues_map

# Data Wrangling
In this section I will clean data

In [74]:
df['CAP']=''
for i,n in zip(df['Address'],df.index):
    for s in i.split(','):
        if s.find('50')>0:
            df['CAP'][n]=s

df['Price']=df['Price'].str.replace(u"Ã¢ÂÂ¬", u"€")
df['Restaurant']=df['Restaurant'].str.replace(u"ÃÂ¨", u"è")
df.drop(columns=['Range_price','Site'], axis=1, inplace=True)
df['Price'].fillna(str(df['Price'].mode()[0]),inplace=True)
df.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Restaurant,Ranking,Reviews,Price,Cousine_type,Address,Phone,Area,Near,Url,Latitude,Longitude,CAP
0,Caffè Italiano,"4,0",1.083 recensioni,€€-€€€,"Italiana, Pizza, Mediterranea, Toscana, Italia...","Via Isola delle Stinche, 11R-13R, 50122, Firen...",+39 055 289080,Duomo,"0,3 km da Piazza del Duomo",https://www.tripadvisor.it/Restaurant_Review-g...,43.770131,11.260304,50122
1,Gustarium,"5,0",1.990 recensioni,€,"Pizza, Mediterranea, Italiana, Fast food, Salu...","Via Dei Cimatori 24/r, 50122, Firenze Italia",+39 055 283469,Duomo,"0,1 km da Palazzo Vecchio",https://www.tripadvisor.it/Restaurant_Review-g...,43.770654,11.255570,50122
2,Degusteria Italiana agli Uffizi,"5,0",689 recensioni,€€-€€€,"Mediterranea, Salutistica, Wine Bar, Italiana,...",Via Lambertesca 7r A Pochi Metri Dal Museo Uff...,+39 055 493 9867,Centro storico di Firenze,"0,1 km da Gallerie Degli Uffizi",https://www.tripadvisor.it/Restaurant_Review-g...,43.768834,11.254358,50122
3,180 Grammi Burgers & Beers,"5,0",80 recensioni,€€-€€€,Italiana,"Via Pietrapiana 11/ R Via Pietrapiana 11/ R, 5...",+39 055 386 0476,Santa Croce,"0,6 km da Piazza del Duomo",https://www.tripadvisor.it/Restaurant_Review-g...,43.771410,11.265810,50121
4,Pitti Express,"5,0",85 recensioni,€,"CaffÃÂ¨, Wine Bar, Toscana, Salutistica, Fast...","Via Romana 40/r, 50125, Firenze Italia",+39 055 627 8178,Oltrarno,"0,6 km da Ponte Vecchio",https://www.tripadvisor.it/Restaurant_Review-g...,43.763190,11.244220,50125


I dropped that restaurant for which I was unable to scrape cousine type

In [75]:
import numpy as np
df=df.dropna(subset=['Cousine_type'])
#df['CAP']=df['CAP'].str.replace('',np.nan)
df=df.dropna(subset=['CAP'])
df['CAP']=df['CAP'].str.replace('50135 Settignano','50135')
df['CAP'].value_counts()

 50122    387
 50123    317
 50125    124
 50129     85
 50124     75
 50121     73
           70
 50127     39
 50100     38
 50142     22
 50144     22
 50126     20
 50136     17
 50137     15
 50134     14
 50143     12
 50132     11
 50135     10
 50131      8
 50133      7
 50141      7
 50139      6
 50145      3
 50014      2
 50063      1
 50026      1
 50023      1
 50041      1
 50060      1
Name: CAP, dtype: int64

In [76]:
dic={'€':1,'€-€€':2,'€€':3,'€€-€€€':4,'€€€':5,'€€€-€€€€':6,'€€€€':7,'€€€€-€€€€€':8,'€€€€€':9}
df['Price']=df['Price'].map(dic)
df['Reviews']=df['Reviews'].str.replace(' recensioni','')
df=df[['Restaurant','Ranking','Reviews','Price','Cousine_type','Latitude','Longitude','CAP']]
df.head()

,Restaurant,Ranking,Reviews,Price,Cousine_type,Latitude,Longitude,CAP
0,Caffè Italiano,"4,0",1.083,4,"Italiana, Pizza, Mediterranea, Toscana, Italia...",43.770131,11.260304,50122
1,Gustarium,"5,0",1.990,1,"Pizza, Mediterranea, Italiana, Fast food, Salu...",43.770654,11.255570,50122
2,Degusteria Italiana agli Uffizi,"5,0",689,4,"Mediterranea, Salutistica, Wine Bar, Italiana,...",43.768834,11.254358,50122
3,180 Grammi Burgers & Beers,"5,0",80,4,Italiana,43.771410,11.265810,50121
4,Pitti Express,"5,0",85,1,"CaffÃÂ¨, Wine Bar, Toscana, Salutistica, Fast...",43.763190,11.244220,50125


In [77]:
df.groupby('CAP').count().head()

,Restaurant,Ranking,Reviews,Price,Cousine_type,Latitude,Longitude
CAP,,,,,,,
,70,70,70,70,70,70,70
50014,2,2,2,2,2,2,2
50023,1,1,1,1,1,1,1
50026,1,1,1,1,1,1,1
50041,1,1,1,1,1,1,1


# Analyzing venues
In this section I'll analyze nearby venues filtering for restaurants.
For one hot encoding I assume that the firs label in 'Cousine_type' is the most relevant for the restaurant. For this reason I'll select only the first label.

In [78]:
df['Cousine_type']=df['Cousine_type'].apply(lambda x: x.split(',')[0])

In [80]:
# one hot encoding
florence_onehot = pd.get_dummies(df[['Cousine_type']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
florence_onehot['CAP'] =df['CAP'] 

# move neighborhood column to the first column
fixed_columns = [florence_onehot.columns[-1]] + list(florence_onehot.columns[:-1])
florence_onehot = florence_onehot[fixed_columns]

florence_onehot.head()

,CAP,Americana,Araba,Armena,Asiatica,Bar,Barbecue,Birreria,Birrerie con ristorante,Brasiliana,...,Pesce,Pizza,Pub,Ristoranti con bar,Salutistica,Spagnola,Steakhouse,Tailandese,Toscana,Wine Bar
0,50122,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,50122,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,50122,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,50121,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,50125,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In the Above tabl I grouped all cousins_type divided for CAP.
In the table below is shown the frequency of each restaurant in each CAP area

In [83]:
florence_grouped = florence_onehot.groupby('CAP').mean().reset_index()[1:].reset_index(drop=True)
florence_grouped.head()

,CAP,Americana,Araba,Armena,Asiatica,Bar,Barbecue,Birreria,Birrerie con ristorante,Brasiliana,...,Pesce,Pizza,Pub,Ristoranti con bar,Salutistica,Spagnola,Steakhouse,Tailandese,Toscana,Wine Bar
0,50014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,50023,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,50041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,50060,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Here below I reported the top five venues per each CAP:

In [84]:
num_top_venues = 5

for hood in florence_grouped['CAP']:
    print("----"+str(hood)+"----")
    temp = florence_grouped[florence_grouped['CAP'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- 50014----
             venue  freq
0         Italiana   1.0
1        Americana   0.0
2            Pesce   0.0
3  Italiana (nord)   0.0
4         Libanese   0.0


---- 50023----
             venue  freq
0         Italiana   1.0
1        Americana   0.0
2            Pesce   0.0
3  Italiana (nord)   0.0
4         Libanese   0.0


---- 50026----
             venue  freq
0         Italiana   1.0
1        Americana   0.0
2            Pesce   0.0
3  Italiana (nord)   0.0
4         Libanese   0.0


---- 50041----
             venue  freq
0         Italiana   1.0
1        Americana   0.0
2            Pesce   0.0
3  Italiana (nord)   0.0
4         Libanese   0.0


---- 50060----
             venue  freq
0         Italiana   1.0
1        Americana   0.0
2            Pesce   0.0
3  Italiana (nord)   0.0
4         Libanese   0.0


---- 50063----
             venue  freq
0         Italiana   1.0
1        Americana   0.0
2            Pesce   0.0
3  Italiana (nord)   0.0
4         Libanese   0.0


In [86]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Finally here below are showed the most ten Cousine_type for each CAP area.

In [94]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['CAP']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['CAP'] = florence_grouped['CAP']

for ind in np.arange(florence_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(florence_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted


,CAP,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,50014,Italiana,Wine Bar,Internazionale,Giapponese,Fast food,Europea,Etiope,Contemporanea,Cinese,Cibo di strada
1,50023,Italiana,Wine Bar,Internazionale,Giapponese,Fast food,Europea,Etiope,Contemporanea,Cinese,Cibo di strada
2,50026,Italiana,Wine Bar,Internazionale,Giapponese,Fast food,Europea,Etiope,Contemporanea,Cinese,Cibo di strada
3,50041,Italiana,Wine Bar,Internazionale,Giapponese,Fast food,Europea,Etiope,Contemporanea,Cinese,Cibo di strada
4,50060,Italiana,Wine Bar,Internazionale,Giapponese,Fast food,Europea,Etiope,Contemporanea,Cinese,Cibo di strada
5,50063,Italiana,Wine Bar,Internazionale,Giapponese,Fast food,Europea,Etiope,Contemporanea,Cinese,Cibo di strada
6,50100,Italiana,Toscana,Steakhouse,Pizza,Giapponese,Bar,Cinese,Wine Bar,Cibo di strada,Europea
7,50121,Italiana,Mediterranea,Steakhouse,Salutistica,Armena,Asiatica,Pesce,Giapponese,Wine Bar,Europea
8,50122,Italiana,Steakhouse,Pizza,Toscana,Mediterranea,Pesce,Americana,Fast food,Giapponese,CaffÃÂ¨
9,50123,Italiana,Steakhouse,Giapponese,Cinese,Pesce,Europea,Indiana,Toscana,Asiatica,Pizza


With this data the client will have a good overview of the restaurants in different areas of Florence. For sure additional analysis could be done but I think the work done till now is in the scope of the assignment.